In [ ]:
import random
import json
from itertools import permutations
import copy

NUMBER_OF_PLAYERS = int(input("Total Number of Players: "))
CUT_NUMBER = int(input("Cut Number: "))
default_trials = 1000
TOTAL_TRIALS = int(input(f"Total Number of Trials (Default: {default_trials}):") or default_trials)

while CUT_NUMBER > NUMBER_OF_PLAYERS/2:
  print("Cut Number must be greater than 2 * Number of Players")
  CUT_NUMBER = int(input("Cut Number: "))

def determine_number_of_rounds(number_of_players):
  power_of_two = 1
  exponent = 0
  while power_of_two < number_of_players:
      power_of_two = 2 ** exponent
      exponent += 1

  num_rounds = exponent - 1

  return num_rounds

NUMBER_OF_ROUNDS = determine_number_of_rounds(NUMBER_OF_PLAYERS)

print("Total Number of Rounds:", NUMBER_OF_ROUNDS)

def create_player_win_loss_storage(number_of_players, num_rounds):
  player_win_loss_storage = {}

  for i in range(number_of_players):
    if i < number_of_players / 2:
      player_win_loss_storage[i] = {"Win_Loss_Schedule": [1]}
    else:
      player_win_loss_storage[i] = {"Win_Loss_Schedule": [0]}

  return player_win_loss_storage

def generate_win_storage(player_win_loss_storage,round_number):
  id_win_storage = {}
  for i in range(round_number):
    id_win_storage[i] = []

  for player_id in player_win_loss_storage.keys():
    wins = player_win_loss_storage[player_id]["Win_Loss_Schedule"].count(1)
    id_win_storage[wins].append(player_id)

  return id_win_storage

def update_players_win_loss(player_win_loss_storage,round_number):
  if round_number < 2:
    return

  id_win_storage = generate_win_storage(player_win_loss_storage,round_number)

  #Play Round, Assumes Pair Downs always Win
  pair_up_win_passed = True
  pair_up_win_passed_win_count = 0
  for win_count in range(round_number):
    player_count = 0
    total_player_count = len(id_win_storage[win_count])
    random.shuffle(id_win_storage[win_count])
    if total_player_count % 2 == 0 or total_player_count == 1:
      for player_id in id_win_storage[win_count]:
        if player_count < total_player_count / 2:
          player_win_loss_storage[player_id]["Win_Loss_Schedule"].append(1)
        else:
          player_win_loss_storage[player_id]["Win_Loss_Schedule"].append(0)
        player_count += 1
    else:
      for player_id in id_win_storage[win_count]:
        if player_count == 0 and pair_up_win_passed:
          player_win_loss_storage[player_id]["Win_Loss_Schedule"].append(0)
          pair_up_win_passed = False
          pair_up_win_passed_win_count = win_count
        elif player_count < total_player_count / 2:
          player_win_loss_storage[player_id]["Win_Loss_Schedule"].append(1)
        else:
          player_win_loss_storage[player_id]["Win_Loss_Schedule"].append(0)

        if pair_up_win_passed_win_count != win_count and pair_up_win_passed:
          pair_up_win_passed = True

        player_count += 1

  return player_win_loss_storage

def play_tournament(player_win_loss_storage,number_of_players,number_of_rounds):
  for round_number in range(2,number_of_rounds+1):
    player_win_loss_storage =  update_players_win_loss(player_win_loss_storage,round_number)
  return player_win_loss_storage

def calculate_CTB_For_All_Players(player_win_loss_storage,number_of_rounds):
  CTB_Storage = {}
  denominator = 0
  for round in range(1,number_of_rounds+1):
    denominator = denominator + round * (0.25 ** (number_of_rounds - round))

  for player_id in player_win_loss_storage.keys():
    nominator = 0
    for round in range(1,number_of_rounds+1):
      wins_at_round = player_win_loss_storage[player_id]["Win_Loss_Schedule"][:round].count(1)
      nominator = nominator + wins_at_round * (0.25 ** (number_of_rounds - round))
    CTB_Storage[player_id] = {"CTB_Score": nominator / denominator, "Wins": player_win_loss_storage[player_id]["Win_Loss_Schedule"].count(1)}

  return CTB_Storage

def convert_schedule_to_string(player_win_loss_schedule):
  schedule_string = ""
  for win in player_win_loss_schedule:
    if win == 1:
      schedule_string = schedule_string + "W"
    else:
      schedule_string = schedule_string + "L"
  return schedule_string

def convert_schedule_str_to_list(schedule_string):
  schedule_list = []
  for char in schedule_string:
    if char == "W":
      schedule_list.append(1)
    else:
      schedule_list.append(0)
  return schedule_list

def determine_tie_breakers(player_ids,ctb_storage,allowed_amount):
  remaining_players = []
  ctbs_unsorted = []
  ctb_counts = {}
  for player_id in player_ids:
    if not ctb_storage[player_id]["CTB_Score"] in ctbs_unsorted:
      ctbs_unsorted.append(ctb_storage[player_id]["CTB_Score"])
      ctb_counts[ctb_storage[player_id]["CTB_Score"]] = 1
    else:
      ctb_counts[ctb_storage[player_id]["CTB_Score"]] += 1
  ctbs_sorted = sorted(ctbs_unsorted,reverse=True)
  allowed_ctbs = ctbs_sorted[:allowed_amount]
  used_ctbs = []

  allowed_idx = len(allowed_ctbs)-1
  idx_hits = 0
  while len(remaining_players) < allowed_amount:
    for player_id in player_ids:
      if ctb_storage[player_id]["CTB_Score"] == allowed_ctbs[allowed_idx]:
        used_ctbs.append(ctb_storage[player_id]["CTB_Score"])
        remaining_players.append(player_id)
        idx_hits += 1
        if idx_hits >= ctb_counts[ctb_storage[player_id]["CTB_Score"]]:
          idx_hits = 0
          allowed_idx -= 1
          if allowed_idx < 0:
            allowed_idx = 0

  lowest_ctb = ctbs_sorted[0]

  return remaining_players,ctb_counts,lowest_ctb

def update_schedule_cuts(player_win_loss_storage,ctb_stroage,number_of_rounds,cut_num,schedule_cut_storage):
  win_storage = generate_win_storage(player_win_loss_storage,number_of_rounds+1)
  total_selected = 0
  for win_count in reversed(range(number_of_rounds+1)):
    if total_selected < cut_num and len(win_storage[win_count]) <= cut_num - total_selected:
      for player_id in win_storage[win_count]:
        schedule_cut_storage[convert_schedule_to_string(player_win_loss_storage[player_id]["Win_Loss_Schedule"])] += 1
        total_selected += 1
    elif total_selected < cut_num:
      remaining_players,ctb_counts,found_lowest_ctb = determine_tie_breakers(win_storage[win_count],ctb_stroage,cut_num - total_selected)
      for player_id in remaining_players:
        schedule_cut_storage[convert_schedule_to_string(player_win_loss_storage[player_id]["Win_Loss_Schedule"])] += 1 / ctb_counts[ctb_stroage[player_id]["CTB_Score"]]
        total_selected += 1

  return schedule_cut_storage

#MAIN
SCHEDULE_CUT_STORAGE = {}

inital_all_losses = "L" * NUMBER_OF_ROUNDS

SCHEDULE_CUT_STORAGE[str(inital_all_losses)] = 0

for num_wins in range(1,NUMBER_OF_ROUNDS):
  base_wins = "L" * (NUMBER_OF_ROUNDS - num_wins) + "W" * num_wins
  premutations_of_wins = [''.join(p) for p in permutations(base_wins)]
  for permutation in premutations_of_wins:
    SCHEDULE_CUT_STORAGE[str(permutation)] = 0

inital_all_wins = "W" * NUMBER_OF_ROUNDS

SCHEDULE_CUT_STORAGE[str(inital_all_wins)] = 0

SCHEDULE_PERCENT_RESULTS = copy.deepcopy(SCHEDULE_CUT_STORAGE)

SCHEDULE_OCCURENCES = copy.deepcopy(SCHEDULE_CUT_STORAGE)

#Do iterations of the same tornament over and over and calculate number of times WL schedule makes it:

for i in range(TOTAL_TRIALS):
  PLAYER_WIN_LOSS_STORAGE = create_player_win_loss_storage(NUMBER_OF_PLAYERS, NUMBER_OF_ROUNDS)
  PLAYER_WIN_LOSS_STORAGE = play_tournament(PLAYER_WIN_LOSS_STORAGE,NUMBER_OF_PLAYERS,NUMBER_OF_ROUNDS)
  CTB_STORAGE = calculate_CTB_For_All_Players(PLAYER_WIN_LOSS_STORAGE,NUMBER_OF_ROUNDS)
  SCHEDULE_CUT_STORAGE = update_schedule_cuts(PLAYER_WIN_LOSS_STORAGE,CTB_STORAGE,NUMBER_OF_ROUNDS,CUT_NUMBER,SCHEDULE_CUT_STORAGE)
  for player_id in PLAYER_WIN_LOSS_STORAGE.keys():
    SCHEDULE_OCCURENCES[convert_schedule_to_string(PLAYER_WIN_LOSS_STORAGE[player_id]["Win_Loss_Schedule"])] += 1

for schedule in SCHEDULE_CUT_STORAGE.keys():
  SCHEDULE_PERCENT_RESULTS[schedule] = 100 * SCHEDULE_CUT_STORAGE[schedule] / SCHEDULE_OCCURENCES[schedule]

ctb_index = {}

for schedule in SCHEDULE_PERCENT_RESULTS.keys():
  ctb_score = calculate_CTB_For_All_Players({schedule:{"Win_Loss_Schedule":convert_schedule_str_to_list(schedule)}},NUMBER_OF_ROUNDS)
  ctb_index[schedule] = ctb_score[schedule]["CTB_Score"]

#Sanity Fixing the Results
for schedule in SCHEDULE_PERCENT_RESULTS.keys():
  for schedule_ctb in ctb_index.keys():
    if ctb_index[schedule] > ctb_index[schedule_ctb] and SCHEDULE_PERCENT_RESULTS[schedule] < SCHEDULE_PERCENT_RESULTS[schedule_ctb]:
      #SCHEDULE_PERCENT_RESULTS[schedule] = SCHEDULE_PERCENT_RESULTS[schedule_ctb]
      pass

print("--------------------------------------------------------------------------------------------------------------------------------------------------------------")
print("Here are the prectentage of times a Win Loss Schedule has made the cut to",CUT_NUMBER,"out of",TOTAL_TRIALS,"simulated tournaments!!!")
print("Disclaimer: Some results may not be 100% accurate due the random nature of pairings and sample size and the fact this simulation assumes pair downs always win.")
print("Disclaimer: This is not an endorsement to stop playing at anypoint, feel free to play a tournament out as this simulation only take into account CTB (Cummalitive Tie Breaker) and nothing beyond that.")
print("NOTE: These results are adjusted such that any Win Loss Schedule with a CTB rating higher than a lower CTB rating has its precentage elevated to the lower CTBs")

#Pretty Print Final Results
for schedule in SCHEDULE_PERCENT_RESULTS.keys():
  if SCHEDULE_PERCENT_RESULTS[schedule] > 0:
    print(schedule,":",SCHEDULE_PERCENT_RESULTS[schedule])




Total Number of Players: 64
Cut Number: 8
Total Number of Trials (Default: 1000):
Total Number of Rounds: 6
--------------------------------------------------------------------------------------------------------------------------------------------------------------
Here are the prectentage of times a Win Loss Schedule has made the cut to 8 out of 1000 simulated tournaments!!!
Disclaimer: Some results may not be 100% accurate due the random nature of pairings and sample size and the fact this simulation assumes pair downs always win.
Disclaimer: This is not an endorsement to stop playing at anypoint, feel free to play a tournament out as this simulation only take into account CTB (Cummalitive Tie Breaker) and nothing beyond that.
NOTE: These results are adjusted such that any Win Loss Schedule with a CTB rating higher than a lower CTB rating has its precentage elevated to the lower CTBs
WLWWWL : 0.5035246727089626
WWLWWL : 2.791625124626121
WWWLWL : 19.556451612903242
WWWWLL : 78.31813